In [ ]:
!pip install dlt[duckdb]

In [2]:
import dlt
print("dlt version:", dlt.__version__)

dlt version: 1.6.1


In [16]:
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator
import duckdb
from google.colab import data_table
import json
import requests

In [31]:
@dlt.resource
def ny_taxi():
    base_url = "https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api"
    page = 1

    while True:
        params = {"page": page}
        response = requests.get(base_url, params=params)
        page_data = response.json()
        if not page_data:
            break
        yield page_data
        page += 1


In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_pipeline_zoomcamp",
    destination="duckdb",
    dataset_name="ny_taxi_data_api"
)

load_info = pipeline.run(ny_taxi)
print("Data Load Info:", load_info)

In [27]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

conn.sql("DESCRIBE").df()

tables = conn.sql("SHOW TABLES").df()

print(tables)

num_tables = len(tables)
print(f"Number of tables: {num_tables}")


                  name
0           _dlt_loads
1  _dlt_pipeline_state
2         _dlt_version
3              ny_taxi
Number of tables: 4


In [35]:
df = pipeline.dataset(dataset_type="default").ny_taxi.df()

print(len(df))

10000


In [32]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM ny_taxi;
            """
        )
    # Prints column values of the first row
    print(res)

[(12.3049,)]
